In [20]:
# Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [24]:
# Target User Accounts
news_org = ['@BBC', '@CBS', '@CNN', '@FoxNews', '@nytimes']

# List for dictionaries of results
sentiment =[]

# Counter
counter = 1

#  Loop through tweets (total 100 tweets)

for news in news_org:
    
    for item in tweepy.Cursor(api.user_timeline, id=news).items(100):
        
        tweet = json.dumps(item._json, indent=3)
        tweet = json.loads(tweet)
        text = tweet['text']
        
        # Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(text)["compound"]
        pos = analyzer.polarity_scores(text)["pos"]
        neu = analyzer.polarity_scores(text)["neu"]
        neg = analyzer.polarity_scores(text)["neg"]
        
        # Add sentiments for each tweet into a list
        news_dict = {
            'source':news,
            'date': tweet["created_at"],
            'text':text,
            'compound': compound,
            'positive': pos,
            'negative': neg,
            'tweets ago': counter}
        
        sentiment.append(news_dict)
        
        # Add to counter 
        counter +=1

In [22]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiment)
sentiments_pd.head()

,compound,date,negative,positive,source,text,tweets ago
0,0.2263,Thu Sep 27 17:02:07 +0000 2018,0.089,0.155,@BBC,This is the quiz where you have to play the an...,1
1,-0.1280,Thu Sep 27 16:03:09 +0000 2018,0.067,0.000,@BBC,Got a bag of frozen peas lurking at the back o...,2
2,-0.5574,Thu Sep 27 15:31:07 +0000 2018,0.194,0.000,@BBC,"John Cunliffe, creator of Postman Pat and Rosi...",3
3,0.5719,Thu Sep 27 15:06:35 +0000 2018,0.000,0.171,@BBC,Flying squirrels are secretive creatures but i...,4
4,-0.4201,Thu Sep 27 14:03:05 +0000 2018,0.117,0.000,@BBC,"""It's very difficult not to tie your self-wort...",5


In [23]:
# Create plot
